## 1.7 Solving the N-Puzzle Problem

The objective of this exercise is the application of search methods, with emphasis on informed
search methods and the A\* algorithm, to solve the well-known N-Puzzle problem. The desired
objective self for the puzzle is as follows (0 represents the empty space):

<table>
<tr><th>9Puzzle</th><th>16Puzzle</th></tr>
<tr>
<td>

|     |     |     |
| --- | --- | --- |
| 1   | 2   | 3   |
| 4   | 5   | 6   |
| 7   | 8   | 0   |


</td>
<td>

|     |     |     |     |
| --- | --- | --- | --- |
| 1   | 2   | 3   | 4   |
| 5   | 6   | 7   | 8   |
| 9   | 10  | 11  | 12  |
| 13  | 14  | 15  | 0   |

</td>
</tr>
</table>

Starting from a given initial state, the goal is to determine which operations to perform to
solve the puzzle, reaching the desired objective self.


In [1]:
from copy import deepcopy
from collections import deque


# definition of the problem
class NPuzzleState:

    def __init__(self, board, move_history=[]):
        # board(list[list[int]]) - the state of the board
        # move_history(list[list[list[int]]]) - the history of the moves up until this state
        self.board = deepcopy(board)
        (self.blank_row, self.blank_col) = self.find_blank()

        # create an empty array and append move_history
        self.move_history = [] + move_history + [self.board]

    def children(self):
        # returns the possible moves
        functions = [self.up, self.down, self.left, self.right]

        children = []
        for func in functions:
            child = func()
            if child:
                children.append(child)

        return children

    def find_blank(self):
        # finds the blank row and col
        for row in range(len(self.board)):
            for col in range(len(self.board[0])):
                if self.board[row][col] == 0:
                    return (row, col)

    def move(func):
        # decorator function to add to history everytime a move is made
        # functions with @move will apply this decorator
        def wrapper(self):
            state = NPuzzleState(self.board, self.move_history)
            value = func(state)
            if value:
                return state
            else:
                return None

        return wrapper

    @move
    def up(self):
        # moves the blank upwards
        if self.blank_row == 0:
            return False
        else:
            self.board[self.blank_row][self.blank_col] = self.board[self.blank_row - 1][self.blank_col]
            self.board[self.blank_row - 1][self.blank_col] = 0
            self.blank_row -= 1
            return True

    @move
    def down(self):
        # moves the blank downwards
        if self.blank_row == len(self.board) - 1:
            return False
        else:
            self.board[self.blank_row][self.blank_col] = self.board[self.blank_row + 1][self.blank_col]
            self.board[self.blank_row + 1][self.blank_col] = 0
            self.blank_row += 1
            return True

    @move
    def left(self):
        # moves the blank left
        if self.blank_col == 0:
            return False
        else:
            self.board[self.blank_row][self.blank_col] = self.board[self.blank_row][self.blank_col - 1]
            self.board[self.blank_row][self.blank_col - 1] = 0
            self.blank_col -= 1
            return True

    @move
    def right(self):
        # moves the blank right
        if self.blank_col == len(self.board[0]) - 1:
            return False
        else:
            self.board[self.blank_row][self.blank_col] = self.board[self.blank_row][self.blank_col + 1]
            self.board[self.blank_row][self.blank_col + 1] = 0
            self.blank_col += 1
            return True

    def is_complete(self):
        # checks if the board is complete
        for row in range(len(self.board)):
            for col in range(len(self.board[0])):
                if self.board[row][col] != row * len(self.board[0]) + col + 1 and self.board[row][col] != 0:
                    return False
        return True

    def __hash__(self):
        # to be able to use the state in a set
        return hash(str([item for sublist in self.board for item in sublist]))

    def __eq__(self, other):
        # compares the two matrices
        return [item for sublist in self.board for item in sublist] == [item for sublist in other.board for item in sublist]

def print_sequence(sequence):
    print("Steps:", len(sequence) - 1)
    # prints the sequence of states
    for state in sequence:
        for row in state:
            print(row)
        print()


def problems():
    return (
        NPuzzleState([[1, 2, 3], [5, 0, 6], [4, 7, 8]]),
        NPuzzleState([[1, 3, 6], [5, 2, 0], [4, 7, 8]]),
        NPuzzleState([[1, 6, 2], [5, 7, 3], [0, 4, 8]]),
        NPuzzleState([[5, 1, 3, 4], [2, 0, 7, 8], [
                     10, 6, 11, 12], [9, 13, 14, 15]]),
    )

**b)** Implement code to solve this problem using the “breadth-first” strategy (in this case
identical to "Uniform Cost"). 

In [4]:
def bfs(problem):
    # problem(NPuzzleState) - the initial state
    queue = deque([problem])
    visited = set() # to not visit the same state twice

    while queue:
        node = queue.popleft()   # get first element in the queue
        if node.is_complete():   # check goal state
            return node.move_history
        
        for state in node.children():   # go through next states
            if state not in visited:
                # enqueue the child node
                queue.append(state)
                visited.add(state)
            
    return None

# prints the sequence for the first problem using bfs
print_sequence(bfs(problems()[2]))

Steps: 10
[1, 6, 2]
[5, 7, 3]
[0, 4, 8]

[1, 6, 2]
[5, 7, 3]
[4, 0, 8]

[1, 6, 2]
[5, 0, 3]
[4, 7, 8]

[1, 0, 2]
[5, 6, 3]
[4, 7, 8]

[1, 2, 0]
[5, 6, 3]
[4, 7, 8]

[1, 2, 3]
[5, 6, 0]
[4, 7, 8]

[1, 2, 3]
[5, 0, 6]
[4, 7, 8]

[1, 2, 3]
[0, 5, 6]
[4, 7, 8]

[1, 2, 3]
[4, 5, 6]
[0, 7, 8]

[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

[1, 2, 3]
[4, 5, 6]
[7, 8, 0]



**c)** Implement code to solve this problem using Greedy Search and using the A*
Algorithm.

Suppose the following heuristics for these methods:
- H1 - Number of incorrect placed pieces;
- H2 - Sum of manhattan distances from incorrect placed pieces to their correct places. 

In [36]:
import heapq # we'll be using a heap to store the states

def greedy_search(problem, heuristic):
    # problem (NPuzzleState) - the initial state
    # heuristic (function) - the heuristic function that takes a board (matrix), and returns an integer
    setattr(NPuzzleState, "__lt__", lambda self, other: heuristic(self) < heuristic(other))
    states = [problem]
    visited = set() # to not visit the same state twice
   
    while states:
        node = heapq.heappop(states)
        if node.is_complete():   # check goal state
            return node.move_history
    
        children = node.children()
        children.sort(reverse=True)

        for state in children:
            if state not in visited:
                heapq.heappush(states, state)
                visited.add(state)

    return None

def _preferential_position(number, side):
    # calculates the preferred position of a piece given its number
    # number (int) - the number of the piece
    # side (int) - the size of the side of the board (only for square boards)
    if number == 0:
        # if it is the last piece, it is 0
        row = col = side - 1
    else:
        # otherwise it is sequential, starting at 1
        row = number // side
        col = number % side - 1
    return (row, col)

def h1(state):
    # heuristic function 1
    # returns the number of incorrect placed pieces in the matrix
    board = state.board
    side = len(board) # the size of the side of the board (only for square boards)

    total = 0

    # checks how many pieces are out of place
    for row in range(side):
        for col in range(side):
            if board[row][col] != row * side + col + 1 and board[row][col] != 0:
                total += 1

    return total

def h2(state):
    # heuristic function 2
    # returns the sum of manhattan distances from incorrect placed pieces to their correct places
    board = state.board
    side = len(board) # the size of the side of the board (only for square boards)
    
    total = 0

    # checks the sum of total distance
    for row in range(side):
        for col in range(side):
           pref_position = _preferential_position(board[row][col], side)
           total += abs(pref_position[0] - row) + abs(pref_position[1] - col)

    return total

print('h1')
print_sequence(greedy_search(problems()[2], h1))

print('h2')
print_sequence(greedy_search(problems()[2], h2))


h2
Steps: 18
[1, 6, 2]
[5, 7, 3]
[0, 4, 8]

[1, 6, 2]
[5, 7, 3]
[4, 0, 8]

[1, 6, 2]
[5, 0, 3]
[4, 7, 8]

[1, 6, 2]
[0, 5, 3]
[4, 7, 8]

[1, 6, 2]
[4, 5, 3]
[0, 7, 8]

[1, 6, 2]
[4, 5, 3]
[7, 0, 8]

[1, 6, 2]
[4, 0, 3]
[7, 5, 8]

[1, 0, 2]
[4, 6, 3]
[7, 5, 8]

[1, 2, 0]
[4, 6, 3]
[7, 5, 8]

[1, 2, 3]
[4, 6, 0]
[7, 5, 8]

[1, 2, 3]
[4, 6, 8]
[7, 5, 0]

[1, 2, 3]
[4, 6, 8]
[7, 0, 5]

[1, 2, 3]
[4, 0, 8]
[7, 6, 5]

[1, 2, 3]
[4, 8, 0]
[7, 6, 5]

[1, 2, 3]
[4, 8, 5]
[7, 6, 0]

[1, 2, 3]
[4, 8, 5]
[7, 0, 6]

[1, 2, 3]
[4, 0, 5]
[7, 8, 6]

[1, 2, 3]
[4, 5, 0]
[7, 8, 6]

[1, 2, 3]
[4, 5, 6]
[7, 8, 0]



In [ ]:
def a_star_search(problem, heuristic):
    # problem (NPuzzleState) - the initial state
    # heuristic (function) - the heuristic function that takes a board (matrix), and returns an integer

    # this is very similar to greedy, the difference is that it takes into account the cost of the path so far
    return greedy_search(problem, lambda state: ????????)


print('h1')
print_sequence(a_star_search(problems()[2], h1))

print('h2')
print_sequence(a_star_search(problems()[2], h2))


**d)** Compare the results obtained concerning execution time and memory space occupied
in solving the following problems using the previous methods

<table>
<tr><th>Prob. 1</th><th>Prob. 2</th><th>Prob. 3</th><th>Prob. 4</th></tr>
<tr>
<td>

|     |     |     |
| --- | --- | --- |
| 1   | 2   | 3   |
| 5   | 0   | 6   |
| 4   | 7   | 8   | 

</td>
<td>

|     |     |     |
| --- | --- | --- |
| 1   | 3   | 6   |
| 5   | 2   | 0   |
| 4   | 7   | 8   | 

</td>
<td>

|     |     |     |
| --- | --- | --- |
| 1   | 6   | 2   |
| 5   | 7   | 3   |
| 0   | 4   | 8   | 

</td>
<td>

|     |     |     |     |
| --- | --- | --- | --- |
| 5   | 1   | 3   | 4   |
| 2   | 0   | 7   | 8   |
| 10  | 6   | 11  | 12  |
| 9   | 13  | 14  | 15  |

</td>
</tr>
</table>

In [51]:
import time


for i in range(len(problems())):
    print("------Problem " + str(i+1) + "------\n")

    #BFS
    time_start = time.time()
    steps = len(bfs(problems()[i]))
    time_taken = round(time.time() - time_start,3)

    print("BFS : ")
    print("   ---> Steps : " + str(steps))
    print("   ---> Time : " + str(time_taken) + " seconds")
    print("   ---> Space : " + str(0) + "\n")

    #Greedy (h1)
    time_start = time.time()
    steps = len(greedy_search(problems()[i],h1))
    time_taken = round(time.time() - time_start,3)

    print("Greedy (h1) : ")
    print("   ---> Steps : " + str(steps))
    print("   ---> Time : " + str(time_taken) + " seconds")
    print("   ---> Space : " + str(0) + "\n")

    #Greedy(h2)
    time_start = time.time()
    steps = len(greedy_search(problems()[i],h2))
    time_taken = round(time.time() - time_start,3)

    print("Greedy (h2) : ")
    print("   ---> Steps : " + str(steps))
    print("   ---> Time : " + str(time_taken) + " seconds")
    print("   ---> Space : " + str(0) + "\n")




------Problem 1------

BFS : 
   ---> Steps : 5
   ---> Time : 0.005 seconds
   ---> Space : 0

Greedy (h1) : 
   ---> Steps : 5
   ---> Time : 0.002 seconds
   ---> Space : 0

Greedy (h2) : 
   ---> Steps : 5
   ---> Time : 0.004 seconds
   ---> Space : 0

------Problem 2------

BFS : 
   ---> Steps : 8
   ---> Time : 0.008 seconds
   ---> Space : 0

Greedy (h1) : 
   ---> Steps : 8
   ---> Time : 0.002 seconds
   ---> Space : 0

Greedy (h2) : 
   ---> Steps : 128
   ---> Time : 0.293 seconds
   ---> Space : 0

------Problem 3------

BFS : 
   ---> Steps : 11
   ---> Time : 0.041 seconds
   ---> Space : 0

Greedy (h1) : 
   ---> Steps : 145
   ---> Time : 0.069 seconds
   ---> Space : 0

Greedy (h2) : 
   ---> Steps : 19
   ---> Time : 0.01 seconds
   ---> Space : 0

------Problem 4------

BFS : 
   ---> Steps : 11
   ---> Time : 0.36 seconds
   ---> Space : 0

Greedy (h1) : 
   ---> Steps : 19
   ---> Time : 0.023 seconds
   ---> Space : 0



KeyboardInterrupt: 